In [1]:
# module import
import os
import sys
from datetime import datetime
import pickle

import numpy as np
import PIL.Image
import torch
import torchvision
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel,  get_cnn_features, img_deprocess, get_target_feature_shape
from activation_maximization import generate_preferred

In [2]:
## load network
net = torchvision.models.vgg19(pretrained=True)
net.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [3]:
# image mean and std for pre/de-process image for input network
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

# if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
norm = 255

In [5]:
# create save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_simpleCNN_higher'#__file__.split('.')[0]
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [6]:
# initial image for the optimization

h, w = 224,224
initial_image = np.zeros(( h,w,3),dtype='float32')


initial_input = np.random.randint(0, 256, (h,w,3))

In [7]:
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [8]:

target_layer = "net.features[8]"
target_layer = "classifier[3]"
extract_list = [target_layer]

In [9]:
#targetlayer
exec_str_list = [target_layer]


exec("num_of_ch = net."+target_layer+".weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(0,num_of_ch,step)

ee = get_cnn_features(net,torch.Tensor(initial_input.transpose(2,0, 1)[np.newaxis]), exec_str_list)
feat_num = ee[0].detach().numpy().shape

In [10]:
# options
opts = {
    'img_mean': img_mean, # img_mean to preprocessing input image (the default is [0.485, 0.456, 0.406]) 
    'img_std': img_std,   # img_std to preprocessing input image  (the default is [0.229,0.224,0.225]) 
    'norm': norm,         # if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1 (defalt is 255)
    'bgr': False,         # if True, input color is adjust to BGR input from RGB input. (default False)
    
    'iter_n': 200, # the total number of iterations for gradient descend (defalt is 200)

    'disp_every': 1, # display the information on the terminal for every n iterations (default is 1)

    'save_intermediate': True, # save the intermediate or not (default is None)
    'save_intermediate_every': 10, # save the intermediate for every n iterations (default is 10)
    'save_intermediate_path': save_path, # the path to save the intermediate (default is None)

    'lr_start': 1., # learning rate (default is 1.)
    'lr_end': 1.,   # we can change learning rate linearly setteing these two parameters 

    'momentum_start': 0.001, # gradient with momentum (default is 0.001)
    'momentum_end': 0.001,   # we can change momentum linearly setteing these two parameters too 

    'decay_start': 0.001, # pixel decay for each iteration (default is 0.001)
    'decay_end': 0.001,   # we can also change pixel decay linealy  

    'image_blur': True, # Use image smoothing or not (default is True)
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing (default is 2.5)
    'sigma_end': 0.5,   

    'image_jitter': True, # use image jittering during optimization (default is True)
    'jitter_size': 4,     # the size of jitter (default is 32)
    
    'use_p_norm_reg': False, # use p_norm regularization (default is False)
    'p': 2,

    'use_TV_norm_reg': False, # use total variance norm (default is False)
    'TVbeta1': 1,             # the order of  spatial domain
    'TVbeta2':1.2,            # the order temporal domain (for video input)

    'clip_small_norm': True,   # clip or not the pixels with extreme high or low value (default True) 
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True, # clip or not the poxels with smal contribution norm of RGB channels (default True)
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'initial_input': initial_input, # the initial image for the optimization (setting to None will use random noise as initial image)
    }


In [11]:
channel_list = np.sort(np.random.choice(np.arange(feat_num[1]), 4))

In [12]:
feat_num

(1, 4096)

In [ ]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')
    #crete feature mask
    feature_mask = np.zeros(feat_num)
    feature_mask[0, channel] = 1.
    # activation maximization
    preferred_stim = generate_preferred(net, exec_str_list, feature_mask=feature_mask, **opts)
    
    # if you want to preferred image for unit, not chanel, you can additionaly define feature mask, which is 
    # binaly mask that target unit is 1 the other is 0.
    
    ## target units
    #feat_size = feat_shape
    #y_index = int(feat_size[2]/2) # the unit in the center of feature map
    #x_index = int(feat_size[3]/2) # the unit in the center of feature map
    #feature_mask = np.zeros(feat_size)
    #feature_mask[0,channel,y_index,x_index] = 1
    ## activation maximization
    #preferred_stim = generate_preferred(net, exec_str_list, feature_mask=feature_mask, **opts)
    
    
    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.npy'
    np.save(os.path.join(save_path,save_name), preferred_stim)

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
    # To better display the image, clip pixels with extreme values (0.02% of
    # pixels with extreme low values and 0.02% of the pixels with extreme high
    # values). And then normalise the image by mapping the pixel value to be
    # within [0,255].
    PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                    os.path.join(save_path, save_name))